In [1]:
import os,shutil,sys
sys.path.append('/data/py/lib/') 
import keras
import time
from keras import models
from keras import layers
from keras import optimizers
from keras.applications import VGG16
from keras.applications import xception
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
####定义一些常用的训练调整参数###########
epochs=13          #定义训练轮数
batch_size=20       #每批数量
lock_layer_num=70;   #锁住的层数
lr=1e-4             #学习率
dense_num=256       #连接层数量
pre_train_epochs=1#预训练轮数,0表示不进行预训练
img_height=299      #训练图片高度
img_width=299       #训练图片宽度
is_load_model=False #是否加载自己训练的历史模型
model_path="/data/keras/models/01281643.h"
##########################

base_dir='/data/keras/download/hua'
train_dir=os.path.join(base_dir,'train')
validation_dir=os.path.join(base_dir,'validation')
test_dir=os.path.join(base_dir,'test')


mod_names=["月季花","蝴蝶兰","海棠花","桂花","发财树","米仔兰","一品红","九里香","茶花","巴西铁树"]

mod_num=len(mod_names)


Using TensorFlow backend.


In [ ]:

#使用图片数据增强，降低拟合的有效手段
train_datagen=ImageDataGenerator(  
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

#验证，测试数据不能进行数据增强
test_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical'   
)

validation_generator=test_datagen.flow_from_directory(   
    validation_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator=test_datagen.flow_from_directory(   
    test_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 2990 images belonging to 10 classes.
Found 497 images belonging to 10 classes.
Found 199 images belonging to 10 classes.


In [ ]:
if is_load_model is False:
    # 构建不带分类器的预训练模型
    base_model = xception.Xception(weights="imagenet",include_top=False,input_shape=(img_height,img_width,3))

    # 添加全局平均池化层
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # 添加一个全连接层
    x = Dense(dense_num, activation='relu')(x)

    # 添加一个分类器
    predictions = Dense(mod_num, activation='softmax')(x)

    # 构建我们需要训练的完整模型
    model = Model(inputs=base_model.input, outputs=predictions)

    # 锁住所有 Xception 的卷积层
    for layer in base_model.layers:
        layer.trainable = False

    #预训练
    if pre_train_epochs>0:
        model.compile(optimizer=optimizers.RMSprop(lr=1e-4), loss='categorical_crossentropy',metrics=['acc'])
        history=model.fit_generator(
            train_generator,
            steps_per_epoch=train_generator.n/train_generator.batch_size,
            epochs=pre_train_epochs,
            validation_data=validation_generator,
            validation_steps=validation_generator.n/validation_generator.batch_size
        )

    # 现在顶层应该训练好了，开始微调 Xception的卷积层。
    # 锁住底下的几层，然后训练其余的顶层。
    # 看看每一层的名字和层号，看看我们应该锁多少层呢：
    # for i, layer in enumerate(base_model.layers):
    #    print(i, layer.name)

    # 锁住的层数
    for layer in model.layers[:lock_layer_num]:
       layer.trainable = False
    for layer in model.layers[lock_layer_num:]:
       layer.trainable = True

    # 设置一个很低的学习率，使用 SGD 来微调
    from keras.optimizers import SGD
    model.compile(optimizer=optimizers.RMSprop(lr=lr), loss='categorical_crossentropy',metrics=['acc'])

    # 继续训练模型
    history=model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n/train_generator.batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.n/validation_generator.batch_size
    )
    #保存训练好的模型
    time_t=time.strftime("%m%d%H%M", time.localtime()) 
    model.save('/data/keras/models/%s.h'%time_t)


Epoch 1/1
150/149 [==============================] - 55s 365ms/step - loss: 1.8827 - acc: 0.4230 - val_loss: 1.5936 - val_acc: 0.4628
Epoch 1/13
150/149 [==============================] - 54s 360ms/step - loss: 0.7983 - acc: 0.7457 - val_loss: 0.5390 - val_acc: 0.8290
Epoch 2/13
  3/149 [..............................] - ETA: 44s - loss: 0.4742 - acc: 0.8667

In [ ]:
# history=model.fit_generator(
#     train_generator,
#     steps_per_epoch=train_generator.n/train_generator.batch_size,
#     epochs=5,
#     validation_data=validation_generator,
#     validation_steps=validation_generator.n/validation_generator.batch_size
# )
# #保存训练好的模型
# time_t=time.strftime("%m%d%H%M", time.localtime()) 
# model.save('/data/keras/models/%s.h'%time_t)

In [ ]:
#显示训练过程中精度变化
if is_load_model is False:
    acc=history.history['acc']
    val_acc=history.history['val_acc']
    loss=history.history['loss']
    val_loss=history.history['val_loss']
    epochs=range(1,len(acc)+1)
    plt.plot(epochs,acc,'bo',label='Training acc')
    plt.plot(epochs,val_acc,'b',label='Validation acc')
    plt.legend()
    plt.figure()
    plt.show()
else:
    model=keras.models.load_model("/data/keras/models/01281643.h")

In [ ]:
#测试集评估
test_loss,test_acc=model.evaluate_generator(test_generator,steps=test_generator.n/test_generator.batch_size)
print ("测试集 acc:%f" % (test_acc))

In [ ]:
#显示真实收集的图片的结果
from keras.preprocessing import image
import numpy as np
import cv2
from mytool import MyTool
test_imgs=['/data/test/hua/蝴蝶兰1.jpg',
           '/data/test/hua/蝴蝶兰2.jpg',
           '/data/test/hua/蝴蝶兰3.jpg',
           '/data/test/hua/茶花2.jpg',
           '/data/test/hua/茶花3.jpg',
           '/data/test/hua/茶花4.jpg',
           '/data/test/hua/茶花5.jpg',
           '/data/test/hua/桂花1.jpg',
            '/data/test/hua/桂花2.jpg',
            '/data/test/hua/一品红1.jpg',
           '/data/test/hua/一品红2.jpg',
           '/data/test/hua/一品红3.jpg',
           '/data/test/hua/米仔兰2.jpg',
           '/data/test/hua/米仔兰3.jpg',
           '/data/test/hua/九里香1.jpg',
           '/data/test/hua/九里香2.jpg',
           '/data/test/hua/海棠花2.jpg',
           '/data/test/hua/海棠花3.jpg',
           '/data/test/hua/巴西铁树1.jpg',
           '/data/test/hua/巴西铁树2.jpg',
           '/data/test/hua/发财树1.jpg',
           '/data/test/hua/发财树2.jpg',
           '/data/test/hua/发财树3.jpg',
           '/data/test/hua/月季花3.jpg',
           '/data/test/hua/月季花4.jpg',
           '/data/test/hua/月季花7.jpg',
           '/data/test/hua/月季花8.jpg',
           '/data/test/hua/月季花9.jpg',
           '/data/test/hua/月季花10.jpg',
           '/data/test/hua/月季花11.jpg',
          ]

for img_path in test_imgs:
    #img = image.load_img(img_path, target_size=(img_height, img_width))
    img =cv2.imread(img_path)
#     plt.imshow(img)
#     plt.show()
    img=MyTool.cro_img(img,img_height,img_width)
    plt.imshow(img)
    plt.show()
    
    x = image.img_to_array(img)
    x=x/255
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
    paixu=dict(zip(train_generator.class_indices,preds[0]))
    paixu= sorted(paixu.items(), key=lambda x: x[1], reverse=True)
    print("图片：",img_path)
    print("预测：",paixu)